Mary Vikhreva's [TSNE-versus-PCA](https://www.kaggle.com/vimary/d/abcsds/pokemon/tsne-vs-pca) demonstrates pretty conclusively that it's not possible to distinguish Pokemon type using Pokemon statistics alone. Even though domain knowledge tells us that there are pretty significant structural differences between Pokemon type statistic attribution (Steel gets high Defense, Psychic high Special Attack, etc.), there are far too many classes for naive classification to work.

Seeing this result, I was curious as to which effect is more important to classification difficulty: the sheer number of Pokemon types total, or exceptions to the "usual pattern" of type stat allocation? If we remove the first difficulty by, say, only considering two types at a time, can we find pairs of Pokemon types that are easily distinguishable?

Distinguishing between just two types of Pokemon isn't, admittedly, very practically interesting. But it tells us something interesting about the structure of Pokemon types: how tightly Pokemon are distributed against the "average" Pokemon of their type.

In [ ]:
import pandas as pd
from pandas.tools.plotting import parallel_coordinates
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
pokemon = pd.read_csv('../input/Pokemon.csv')

If we dump our data out onto a [parallel coordinates plot](https://en.wikipedia.org/wiki/Parallel_coordinates), we get what we expect&mdash;a big mess.

In [ ]:
plt.figure(figsize=(12, 5))
parallel_coordinates(pokemon[
        ['Type 1', 'HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']
    ], 'Type 1')

No classifier could possibly make sense of this.

Let's visualize some type pairs instead.

First up, Steel versus Fairy types.

In [ ]:
plt.figure(figsize=(12, 5))
parallel_coordinates(pokemon[pokemon['Type 1'].isin(['Steel', 'Fairy'])][
        ['Type 1', 'HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']
    ], 'Type 1',  colormap=lambda t: 'DarkOrchid' if t == 0 else 'DarkGray')

Domain knowledge tells us that these should be distinguishable, and it appears that they are! Steel Pokemon in general have a much higher Defense stat than Fairy Pokemon do.

Here's another promising type comparison, Psychic and Fighting.

In [ ]:
plt.figure(figsize=(12, 5))
parallel_coordinates(pokemon[pokemon['Type 1'].isin(['Psychic', 'Fighting'])][
        ['Type 1', 'HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']
    ], 'Type 1',  colormap=lambda t: 'PaleVioletRed' if t == 0 else 'FireBrick')

In [ ]:
plt.figure(figsize=(12, 5))
parallel_coordinates(pokemon[pokemon['Type 1'].isin(['Ghost', 'Psychic'])][
        ['Type 1', 'HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']
    ], 'Type 1', colormap=lambda t: 'PaleVioletRed' if t == 0 else 'Indigo')

Psychic Pokemon focus on Sp. Atk while Fighting Pokemon focus on regular Attack, and neither type tends to allocate to the other.

These two represent relatively distinguishable type comparisons. How about one that'd never work? Here's one.

In [ ]:
plt.figure(figsize=(8, 4))
parallel_coordinates(pokemon.groupby('Type 1').mean().reset_index()[
        ['Type 1', 'HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']
    ], 'Type 1')
ax = plt.gca()
handles, labels = ax.get_legend_handles_labels()
ax.legend(handles, labels, loc='upper right', bbox_to_anchor=(1.3, 1))

This is just a big muddle!

If we look at the averages across all types, we see that a few of these types do indeed seem pretty distinct: Steel, Dragon, Flying, and Fairy types all, in aggregate, have structural features in stat allocation unique to their type.

In [ ]:
def predict_primary_type(hp, speed, attack, defense, sp_atk, sp_def, bound=(25, 25)):
    ret = pokemon
    for v in zip(['HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def'], [hp, speed, attack, defense, sp_atk, sp_def]):
        v_low = v[1] - bound[0]
        v_high = v[1] + bound[1]
        ret = ret[(v_low < ret[v[0]]) & (ret[v[0]] < v_high)]
    ret = ret.groupby('Type 1').count()['#']
    return ret / ret.sum()

In [ ]:
predict_primary_type(70, 70, 70, 70, 70, 70)

This data allows us to go the other way and check typing by stat allocation, if we are so inclined, and we are. Here's a type breakdown for Pokemon with stats matching an "average" Pokemon (Bulbasaur is in this list, for example).

In [ ]:
steel_avg = pokemon.groupby('Type 1').mean().ix['Steel'][['HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']].values

The top two classes cover just a quarter of cases!

Meanwhile, even more troublingly, the most likely Pokemon type near the Steel allocation average is the Bug type!

There are simply a lot more Bug type than Steel type Pokemon, and some of them fall into our "averagely a Steel type" window.

In [ ]:
predict_primary_type(*steel_avg)

Indeed, Pokemon types in general own very little of their average "space". The top contender is normal types, again probably mostly because that's simply the most common type in the game.

In [ ]:
for _type in labels:
    type_avg = pokemon.groupby('Type 1').mean().ix[_type][['HP', 'Speed', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def']].values
    print(_type.ljust(10), predict_primary_type(*type_avg)[_type])

Well, this is a dead end.

Let's see what happens when we try to run a classifier. We will preprocess with [principal components analysis](https://en.wikipedia.org/wiki/Principal_component_analysis) to afix the variables of interest, then run a [support vector machine](https://en.wikipedia.org/wiki/Support_vector_machine) on the result.

As you in the explained variable output below, most of the variability in the data in our test case (Steel versus Fairy) is explained by two features post-PCA, so we'll drop the others and focus on those. This allows us to easily plot what we're doing&mdash;trying to draw a line seperating our two classes in two-dimensional space.

In [ ]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
features = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
steel_or_fairy = pokemon[pokemon['Type 1'].isin(['Steel', 'Fairy'])][features]
type_actual = pokemon[pokemon['Type 1'].isin(['Steel', 'Fairy'])]['Type 1']

In [ ]:
df_norm = steel_or_fairy.copy()
df_norm[features] = StandardScaler().fit(steel_or_fairy[features]).transform(steel_or_fairy[features])

steel_or_fairy_pca = PCA()
pca_outcomes = steel_or_fairy_pca.fit_transform(df_norm[features])

In [ ]:
steel_or_fairy_pca.explained_variance_ 

In [ ]:
first_two_principal_components = pca_outcomes[:,[0,1]]

plt.title("Fairy or Steel Type, Actual")
plt.scatter(first_two_principal_components[:,0], first_two_principal_components[:,1],
            c=['DarkOrchid' if t else 'DarkGray' for t in (type_actual == 'Fairy')])

In [ ]:
from sklearn import svm

clf = svm.SVC(kernel='linear', C=1.0).fit(first_two_principal_components, type_actual)

f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,4))
ax1.set_title('Fairy or Steel, Actual')
ax2.set_title('Fairy or Steel, Predicted')
ax1.scatter(first_two_principal_components[:,0], first_two_principal_components[:,1],
            c=['DarkOrchid' if t else 'DarkGray' for t in (type_actual == 'Fairy')])
ax2.scatter(first_two_principal_components[:,0], first_two_principal_components[:,1],
            c=['DarkOrchid' if t else 'DarkGray' for t in (clf.predict(first_two_principal_components) == 'Fairy')])

Can you see where the classifier drew a line?

This is a pretty good result! But again, remember that Fairy and Steel are probably the most easily co-distinguishable types in the game. Let's formalize this procedure and look at a handful of other results.

In [ ]:
import matplotlib.patches as mpatches

def pairwise_classify(type_1, type_2):
    features = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
    df = pokemon[pokemon['Type 1'].isin([type_1, type_2])][features]
    df_norm = df.copy()
    df_norm[features] = StandardScaler().fit(df[features]).transform(df[features])

    pca_outcomes = PCA().fit_transform(df_norm[features])
    principal_components = pca_outcomes[:,[0,1]]
    
    y = pokemon[pokemon['Type 1'].isin([type_1, type_2])]['Type 1'].values

    clf = svm.SVC(kernel='linear', C=1.0).fit(principal_components, y)
    
    f, (ax1, ax2) = plt.subplots(1, 2, sharey=True, figsize=(12,4))
    ax1.set_title('{0} or {1}, Actual'.format(type_1, type_2))
    ax2.set_title('{0} or {1}, Predicted'.format(type_1, type_2))
    
    ax1.scatter(principal_components[:,0], principal_components[:,1],
            c=['#b4464b' if t else '#4682b4' for t in (y == type_1)], lw = 0, s=40)
    ax2.scatter(principal_components[:,0], principal_components[:,1],
            c=['#b4464b' if t else '#4682b4' for t in (clf.predict(principal_components) == type_1)], lw = 0, s=40)
    
    red_patch = mpatches.Patch(color='#b4464b', label=type_1)
    blue_patch = mpatches.Patch(color='#4682b4', label=type_2)
    ax1.legend(handles=[red_patch, blue_patch])
    ax2.legend(handles=[red_patch, blue_patch])
    
    plt.show()

In [ ]:
pairwise_classify('Steel', 'Fairy')

This is not as great:

In [ ]:
pairwise_classify('Dragon', 'Ice')

These are disasters; the SVM simply collapses onto a single class!

In [ ]:
pairwise_classify('Fighting', 'Psychic')

In [ ]:
pairwise_classify('Bug', 'Ground')

Let's extend this to all type pairs. First, we define a distinguishability score, which we'll set to None if the result is a failing classification like the last two above. Then we'll dump the resultant data out into a table.

In [ ]:
from sklearn import metrics
import numpy as np

def distinguishability(type_1, type_2):
    features = ['Total', 'HP', 'Attack', 'Defense', 'Sp. Atk', 'Sp. Def', 'Speed']
    df = pokemon[pokemon['Type 1'].isin([type_1, type_2])][features]
    df_norm = df.copy()
    df_norm[features] = StandardScaler().fit(df[features]).transform(df[features])

    pca_outcomes = PCA().fit_transform(df_norm[features])
    principal_components = pca_outcomes[:,[0,1]]
    
    y = pokemon[pokemon['Type 1'].isin([type_1, type_2])]['Type 1'].values

    clf = svm.SVC(kernel='linear', C=1.0).fit(principal_components, y)
        
    conf = metrics.confusion_matrix(y, clf.predict(principal_components))
    
    # If we simply classify all records as one type or the other our classifier has failed to be useful.
    # In that case return np.nan.
    if all(conf[:,0] == [0,0]) or all(conf[:,1] == [0,0]):
        return np.nan
    else:
        return (conf[0][0] + conf[1][1]) / conf.sum()

In [ ]:
distinguishability('Dragon', 'Ice')

In [ ]:
distinguishability('Bug', 'Ice')

In [ ]:
types = np.unique(pokemon['Type 1'])

values = []
for type_1 in types:
    values.append([])
    for type_2 in types:
        if type_1 != type_2:
            values[-1].append(distinguishability(type_1, type_2))
        else:
            values[-1].append(np.nan)

Let's subtract 0.5 from the result to get how-much-better-than-a-coin-flip our classifier did.

In [ ]:
type_distinguishabilities = pd.DataFrame(index=types, columns=types, data=values)
classification_gain = type_distinguishabilities - 0.5

Voila, a big table.

In [ ]:
classification_gain

That's a lot of detail. As you can see, there are many, many cases where the classifier failed to do anything at all. Here's average distinguishability by type:

In [ ]:
classification_gain.fillna(0).mean()

We've found exactly the same types we were thinking about at the beginning of this experiment&mdash;Steel, Dragon, Electric. But now we have numbers to attach to our hypothesis.

Here's a few example classification schemes.

In [ ]:
pairwise_classify('Dragon', 'Normal')

In [ ]:
pairwise_classify('Dragon', 'Steel')

In [ ]:
pairwise_classify('Steel', 'Flying')

In [ ]:
pairwise_classify('Ground', 'Electric')

In conclusion:

**The average Pokemon of a type is not representative of Pokemon of that type as a whole.**